In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import mlflow
import mlflow.sklearn

In [2]:
mlflow.set_tracking_uri("http://localhost:5001")  # 로컬 MLflow experiment 설정
mlflow.set_experiment("core3_degradation_prediction")

MlflowException: API request to http://localhost:5001/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=core3_degradation_prediction (Caused by ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))

In [ ]:
df = pd.read_csv("../data/raw/health_timeseries_core_state.csv")
df["date"] = pd.to_datetime(df["date"])

df = (
    df.rename(columns={
        "user_id": "asset_id",
        "health_state_index": "state_value"
    })
    .sort_values(["asset_id", "date"])
    .reset_index(drop=True)
)

# Core2와 동일한 파생
df["delta_state"] = df.groupby("asset_id")["state_value"].diff()
df["state_lag1"] = df.groupby("asset_id")["state_value"].shift(1)

train_df = df.dropna(subset=["delta_state", "state_lag1"]).reset_index(drop=True)

X = train_df[["state_lag1"]]
y = train_df["delta_state"]

Run 1: Linear Regression (baseline)

In [ ]:
with mlflow.start_run(run_name="linear_regression_baseline"):

    model = LinearRegression()
    model.fit(X, y)

    y_pred = model.predict(X)

    rmse = np.sqrt(mean_squared_error(y, y_pred))   # ← 수정 포인트
    error_std = np.std(y - y_pred)

    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("error_std", error_std)

    mlflow.sklearn.log_model(model, "model")

    print("Run 1 RMSE:", rmse)
    print("Run 1 error_std:", error_std)

2025/12/27 18:49:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run linear_regression_baseline at: http://localhost:5001/#/experiments/159696756529388088/runs/c9bddfe013fd494aabb7c55b5b2a2cf8
🧪 View experiment at: http://localhost:5001/#/experiments/159696756529388088


MlflowException: API request to endpoint /api/2.0/mlflow/logged-models failed with error code 404 != 200. Response body: '<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'

Run 2: 동일 모델, 다른 window
- 모델은 같고, 입력 분포만 다름

In [ ]:
with mlflow.start_run(run_name="linear_regression_short_window"):

    # 최근 데이터만 사용 (예: 최근 50%)
    cutoff = int(len(train_df) * 0.5)
    X_short = X.iloc[cutoff:]
    y_short = y.iloc[cutoff:]

    model = LinearRegression()
    model.fit(X_short, y_short)

    y_pred = model.predict(X_short)

    rmse = mean_squared_error(y_short, y_pred, squared=False)
    error_std = np.std(y_short - y_pred)

    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("data_scope", "recent_half")
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("error_std", error_std)

    mlflow.sklearn.log_model(model, "model")

    print("Run 2 RMSE:", rmse)
    print("Run 2 error_std:", error_std)

### Core 3 결론

- 상태 열화 예측은 단일 성능 수치로 판단할 수 없다.
- 동일한 모델이라도 데이터 조건에 따라 오차 특성이 달라진다.
- 따라서 예측 모델은 **의사결정 입력으로 사용되기 전, 관리·추적되어야 한다**.

이후 Core 단계에서는,
모델 자체보다 **이 예측값을 어떻게 사용하는가**가 더 중요해진다.